In [1]:
import pandas as pd
import csv
from pathlib import Path
import datetime as dt
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import hvplot.pandas as hvplot
%matplotlib inline


In [2]:
tickers_data = Path("/Users/matt/Desktop/senator_trading_performance/senators_fixed_dates_&amounts.csv")
tickers_data_df = pd.read_csv(tickers_data, infer_datetime_format=True, parse_dates=True)

tickers_data_df

,transaction_date,owner,ticker,asset_type,type,amount,state,industry,senator
0,4/18/2023,Spouse,ESS,Stock,Sale (Full),"$15,000",RI,Real Estate Investment Trusts,Sheldon Whitehouse
1,4/18/2023,Self,ESS,Stock,Sale (Full),"$15,000",RI,Real Estate Investment Trusts,Sheldon Whitehouse
2,5/16/2023,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Michael F. Bennet
3,4/4/2023,Spouse,UPS,Stock,Sale (Full),"$15,000",WV,Trucking Freight/Courier Services,Shelley Moore Capito
4,4/4/2023,Spouse,MCD,Stock,Sale (Partial),"$15,000",WV,Restaurants,Shelley Moore Capito
...,...,...,...,...,...,...,...,...,...
8441,8/17/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Sheldon Whitehouse
8442,8/16/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Pat Roberts
8443,8/15/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Rob Portman
8444,8/2/2012,NaN,NaN,PDF Disclosed Filing,NaN,NaN,NaN,NaN,Thomas R. Carper


Find the specific tickers that each senator is trading and bring in data for those tickers

Senator, date, amount, ticker, type

In [3]:
tickers_df_dropped=tickers_data_df.dropna()
tickers_df_dropped.head

<bound method NDFrame.head of      transaction_date   owner ticker asset_type            type     amount  \
0           4/18/2023  Spouse    ESS      Stock     Sale (Full)   $15,000    
1           4/18/2023    Self    ESS      Stock     Sale (Full)   $15,000    
3            4/4/2023  Spouse    UPS      Stock     Sale (Full)   $15,000    
4            4/4/2023  Spouse    MCD      Stock  Sale (Partial)   $15,000    
5            4/4/2023  Spouse    DUK      Stock     Sale (Full)   $50,000    
...               ...     ...    ...        ...             ...        ...   
7627        1/16/2015    Self    NOG      Stock        Purchase   $50,000    
7628         1/6/2015    Self    FCX      Stock     Sale (Full)  $100,000    
7640       12/12/2014  Spouse    XOM      Stock     Sale (Full)   $15,000    
7641       12/19/2014  Spouse     BA      Stock        Purchase   $15,000    
7642       12/12/2014  Spouse    BAC      Stock     Sale (Full)   $15,000    

     state                       

In [4]:
tickers_df=pd.DataFrame(tickers_df_dropped)
# tickers_df

tickers_df=tickers_df[['transaction_date', 'ticker', 'type', 'amount', 'senator']]
display(tickers_df)

,transaction_date,ticker,type,amount,senator
0,4/18/2023,ESS,Sale (Full),"$15,000",Sheldon Whitehouse
1,4/18/2023,ESS,Sale (Full),"$15,000",Sheldon Whitehouse
3,4/4/2023,UPS,Sale (Full),"$15,000",Shelley Moore Capito
4,4/4/2023,MCD,Sale (Partial),"$15,000",Shelley Moore Capito
5,4/4/2023,DUK,Sale (Full),"$50,000",Shelley Moore Capito
...,...,...,...,...,...
7627,1/16/2015,NOG,Purchase,"$50,000",John Hoeven
7628,1/6/2015,FCX,Sale (Full),"$100,000",John Hoeven
7640,12/12/2014,XOM,Sale (Full),"$15,000",Pat Roberts
7641,12/19/2014,BA,Purchase,"$15,000",Pat Roberts


In [5]:
tickers_counts=tickers_df['ticker'].value_counts()
msft_counts=tickers_counts['MSFT']


msft_counts

96

In [6]:
most_common_ticker=tickers_df['ticker'].value_counts().head(5)
most_common_ticker

AAPL    168
MSFT     96
BAC      81
DIS      79
PFE      77
Name: ticker, dtype: int64

Bring in daily close for AAPL for every transaction, iterate through list and bring in daily close price for transaction date and 30 days into the future.

In [7]:
msft_filtered_df=tickers_df[(tickers_df['ticker']=='MSFT') & (tickers_df['amount'])]
# display(msft_filtered_df)

aapl_filtered_df=tickers_df[(tickers_df['ticker']=='AAPL') & (tickers_df['amount'])]
# display(aapl_filtered_df)

bac_filtered_df=tickers_df[(tickers_df['ticker']=='BAC') & (tickers_df['amount'])]
# display(bac_filtered_df)

dis_filtered_df=tickers_df[(tickers_df['ticker']=='DIS') & (tickers_df['amount'])]
# display(dis_filtered_df)

pfe_filtered_df=tickers_df[(tickers_df['ticker']=='PFE') & (tickers_df['amount'])]
# display(pfe_filtered_df)

combined_data=pd.concat([msft_filtered_df, aapl_filtered_df, bac_filtered_df, dis_filtered_df, pfe_filtered_df], axis=0)
display(combined_data)


,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
301,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
309,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
...,...,...,...,...,...
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7326,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


In [8]:
dup_drop=combined_data.drop_duplicates()
dup_drop

,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
323,11/23/2022,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,11/23/2022,MSFT,Purchase,"$15,000",Tommy Tuberville
...,...,...,...,...,...
6963,9/16/2015,PFE,Purchase,"$15,000",Gary C. Peters
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


In [9]:
ready_csv_data=dup_drop.dropna()#.set_index('transaction_date')
display(ready_csv_data)



,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
323,11/23/2022,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,11/23/2022,MSFT,Purchase,"$15,000",Tommy Tuberville
...,...,...,...,...,...
6963,9/16/2015,PFE,Purchase,"$15,000",Gary C. Peters
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


This is the API for stock data that I will concat to the existing df, I will iterate through each row and input the ticker symbol into the API 

Use top 5 tickers invested

In [10]:
# duration=180
# for idx, row in senator_trades_df.iterrows(): 
#     start=row['transaction_date'] # 2020-01-03
#     ticker=row['ticker'] # AAPL
#     df_in_timeframe=yf_data.loc[start:start+duration, ticker]
#     return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
#     senator_trades_df.loc[idx, f'{duration}_days_return']=return_in_timeframe

Concat the 5 tickers close prices into one df and input into the for loop for yf_data

In [11]:
import yfinance as yf
from datetime import datetime, timedelta

# 6 years of data
six_year_end_date = datetime.today()
six_year_start_date = six_year_end_date - timedelta(days=365*9)

# # 30 days of data
# month_end_date = datetime.today()
# month_start_date = end_date - timedelta(days=30)

# # 90 days of data
# three_month_end_date = datetime.today()
# three_month_start_date = end_date - timedelta(days=90)

# # 180 days of data
# six_month_end_date = datetime.today()
# six_month_start_date = six_month_end_date - timedelta(days=180)

msft_tickers = yf.Tickers('MSFT')
msft_data=msft_tickers.tickers['MSFT'].history(start=six_year_start_date, end=six_year_end_date)
msft_ticker_df=pd.DataFrame(msft_data)
msft_ticker_df.index=msft_ticker_df.index.date
# msft_ticker_df

aapl_tickers=yf.Tickers('AAPL')
aapl_data=aapl_tickers.tickers['AAPL'].history(start=six_year_start_date, end=six_year_end_date)
aapl_ticker_df=pd.DataFrame(aapl_data)
aapl_ticker_df.index=aapl_ticker_df.index.date
# aapl_ticker_df


bac_tickers=yf.Tickers('BAC')
bac_data=bac_tickers.tickers['BAC'].history(start=six_year_start_date, end=six_year_end_date)
bac_ticker_df=pd.DataFrame(bac_data)
bac_ticker_df.index=bac_ticker_df.index.date
# bac_ticker_df

dis_tickers=yf.Tickers('DIS')
dis_data=dis_tickers.tickers['DIS'].history(start=six_year_start_date, end=six_year_end_date)
dis_ticker_df=pd.DataFrame(dis_data)
dis_ticker_df.index=dis_ticker_df.index.date
# dis_ticker_df

pfe_tickers=yf.Tickers('PFE')
pfe_data=pfe_tickers.tickers['PFE'].history(start=six_year_start_date, end=six_year_end_date)
pfe_ticker_df=pd.DataFrame(pfe_data)
pfe_ticker_df.index=pfe_ticker_df.index.date
# pfe_ticker_df



In [12]:
msft_filtered_ticker_df=msft_ticker_df[['Close']]
# msft_filtered_ticker_df

aapl_filtered_ticker_df=aapl_ticker_df[['Close']]
# display(aapl_filtered_ticker_df)

bac_filtered_ticker_df=bac_ticker_df[['Close']]
# display(bac_filtered_ticker_df)


dis_filtered_ticker_df=dis_ticker_df[['Close']]
# display(dis_filtered_ticker_df)


pfe_filtered_ticker_df=pfe_ticker_df[['Close']]
# display(pfe_filtered_ticker_df)


ready_ticker_data=pd.concat([msft_filtered_ticker_df, aapl_filtered_ticker_df, bac_filtered_ticker_df, dis_filtered_ticker_df, pfe_filtered_ticker_df], axis=1, keys=['MSFT', 'AAPL', 'BAC', 'DIS']).dropna()
display(ready_ticker_data)

,MSFT,AAPL,BAC,DIS
,Close,Close,Close,Close
2014-08-06,36.666737,21.110390,12.784967,79.690178
2014-08-07,37.087116,21.108152,12.717675,78.696251
2014-08-08,37.061390,21.166241,12.784967,79.929466
2014-08-11,37.061390,21.445505,12.801788,80.527664
2014-08-12,37.335918,21.441038,12.793377,80.260773
...,...,...,...,...
2023-07-28,338.369995,195.830002,31.900000,86.129997
2023-07-31,335.920013,196.449997,32.000000,88.889999
2023-08-01,336.339996,195.610001,31.620001,89.029999


In [13]:
msft_filtered_ticker_df=msft_ticker_df[['Close']]
# msft_filtered_ticker_df

aapl_filtered_ticker_df=aapl_ticker_df[['Close']]
# display(aapl_filtered_ticker_df)

bac_filtered_ticker_df=bac_ticker_df[['Close']]
# display(bac_filtered_ticker_df)


dis_filtered_ticker_df=dis_ticker_df[['Close']]
# display(dis_filtered_ticker_df)


pfe_filtered_ticker_df=pfe_ticker_df[['Close']]
# display(pfe_filtered_ticker_df)


ready_ticker_data=pd.concat([msft_filtered_ticker_df, aapl_filtered_ticker_df, bac_filtered_ticker_df, dis_filtered_ticker_df, pfe_filtered_ticker_df], axis=1, keys=['MSFT', 'AAPL', 'BAC', 'DIS', 'PFE']).dropna()
display(ready_ticker_data)

ready_ticker_data.columns

,MSFT,AAPL,BAC,DIS,PFE
,Close,Close,Close,Close,Close
2014-08-06,36.666737,21.110390,12.784967,79.690178,19.287703
2014-08-07,37.087116,21.108152,12.717675,78.696251,19.124018
2014-08-08,37.061390,21.166241,12.784967,79.929466,19.328621
2014-08-11,37.061390,21.445505,12.801788,80.527664,19.267242
2014-08-12,37.335918,21.441038,12.793377,80.260773,19.151295
...,...,...,...,...,...
2023-07-28,338.369995,195.830002,31.900000,86.129997,36.070000
2023-07-31,335.920013,196.449997,32.000000,88.889999,36.060001
2023-08-01,336.339996,195.610001,31.620001,89.029999,35.610001


MultiIndex([('MSFT', 'Close'),
            ('AAPL', 'Close'),
            ( 'BAC', 'Close'),
            ( 'DIS', 'Close'),
            ( 'PFE', 'Close')],
           )

In [14]:
new_cols=[col for col, close_col in ready_ticker_data.columns]

ready_ticker_data.columns=new_cols

ready_ticker_data.head()

,MSFT,AAPL,BAC,DIS,PFE
2014-08-06,36.666737,21.110390,12.784967,79.690178,19.287703
2014-08-07,37.087116,21.108152,12.717675,78.696251,19.124018
2014-08-08,37.061390,21.166241,12.784967,79.929466,19.328621
2014-08-11,37.061390,21.445505,12.801788,80.527664,19.267242
2014-08-12,37.335918,21.441038,12.793377,80.260773,19.151295


In [15]:
ready_ticker_data.tail()

,MSFT,AAPL,BAC,DIS,PFE
2023-07-28,338.369995,195.830002,31.900000,86.129997,36.070000
2023-07-31,335.920013,196.449997,32.000000,88.889999,36.060001
2023-08-01,336.339996,195.610001,31.620001,89.029999,35.610001
2023-08-02,327.500000,192.580002,31.190001,86.300003,35.349998
2023-08-03,326.660004,191.169998,31.410000,85.489998,35.000000


In [16]:
ready_csv_data.head()

,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
323,11/23/2022,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,11/23/2022,MSFT,Purchase,"$15,000",Tommy Tuberville


In [17]:
complete_df=pd.DataFrame(ready_csv_data)
complete_df

,transaction_date,ticker,type,amount,senator
50,4/3/2023,MSFT,Sale (Partial),"$50,000",Tommy Tuberville
293,11/22/2022,MSFT,Purchase,"$15,000",Dan Sullivan
318,11/30/2022,MSFT,Purchase,"$50,000",Dan Sullivan
323,11/23/2022,MSFT,Sale (Full),"$50,000",Tommy Tuberville
324,11/23/2022,MSFT,Purchase,"$15,000",Tommy Tuberville
...,...,...,...,...,...
6963,9/16/2015,PFE,Purchase,"$15,000",Gary C. Peters
7323,4/29/2015,PFE,Sale (Partial),"$15,000",David Perdue
7325,5/6/2015,PFE,Sale (Partial),"$15,000",David Perdue
7327,5/7/2015,PFE,Sale (Partial),"$15,000",David Perdue


In [18]:
ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])

duration=30
for idx, row in ready_csv_data.iterrows(): 
    start=row['transaction_date'] # 2020-01-03
    ticker=row['ticker'] # AAPL
    
    if ticker not in ready_ticker_data.columns:
        continue
        
    end =start+timedelta(days=duration)
    df_in_timeframe=ready_ticker_data.loc[start:end, ticker]

    
    if df_in_timeframe.empty:
        continue
        
    return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
    ready_csv_data.loc[idx, f'{duration}_days_return']=return_in_timeframe.item()

display(ready_csv_data)

/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/2411737893.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]
/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/2411737893.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]


,transaction_date,ticker,type,amount,senator,30_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746
...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018828
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188


In [19]:
ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])

duration=60
for idx, row in ready_csv_data.iterrows(): 
    start=row['transaction_date'] # 2020-01-03
    ticker=row['ticker'] # AAPL
    
    if ticker not in ready_ticker_data.columns:
        continue
        
    end =start+timedelta(days=duration)
    df_in_timeframe=ready_ticker_data.loc[start:end, ticker]

    
    if df_in_timeframe.empty:
        continue
        
    return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
    ready_csv_data.loc[idx, f'{duration}_days_return']=return_in_timeframe.item()

display(ready_csv_data)

/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/1432870943.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]
/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/1432870943.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]


,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728
...,...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070,0.010806
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800,-0.002605
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018828,-0.002690
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188,-0.009827


In [20]:
ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])

duration=90
for idx, row in ready_csv_data.iterrows(): 
    start=row['transaction_date'] # 2020-01-03
    ticker=row['ticker'] # AAPL
    
    if ticker not in ready_ticker_data.columns:
        continue
        
    end =start+timedelta(days=duration)
    df_in_timeframe=ready_ticker_data.loc[start:end, ticker]

    
    if df_in_timeframe.empty:
        continue
        
    return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
    ready_csv_data.loc[idx, f'{duration}_days_return']=return_in_timeframe.item()

display(ready_csv_data)

/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/679451525.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]
/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/679451525.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]


,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return,90_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258,0.188192
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630,0.055815
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358,-0.019971
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728,0.023115
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728,0.023115
...,...,...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070,0.010806,-0.019880
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800,-0.002605,0.039443
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018828,-0.002690,0.079199
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188,-0.009827,0.075042


In [21]:
ready_csv_data['transaction_date']=pd.to_datetime(ready_csv_data['transaction_date'])

duration=180
for idx, row in ready_csv_data.iterrows(): 
    start=row['transaction_date'] # 2020-01-03
    ticker=row['ticker'] # AAPL
    
    if ticker not in ready_ticker_data.columns:
        continue
        
    end =start+timedelta(days=duration)
    df_in_timeframe=ready_ticker_data.loc[start:end, ticker]

    
    if df_in_timeframe.empty:
        continue
        
    return_in_timeframe=df_in_timeframe.iloc[-1]/df_in_timeframe.iloc[0]-1
    ready_csv_data.loc[idx, f'{duration}_days_return']=return_in_timeframe.item()

display(ready_csv_data)

/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/3204014247.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]
/var/folders/zs/mcg4c63n0vl83nylzs3npqv00000gn/T/ipykernel_57066/3204014247.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_in_timeframe=ready_ticker_data.loc[start:end, ticker]


,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return,90_days_return,180_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258,0.188192,0.139763
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630,0.055815,0.305289
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358,-0.019971,0.310862
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
...,...,...,...,...,...,...,...,...,...
6963,2015-09-16,PFE,Purchase,"$15,000",Gary C. Peters,0.037070,0.010806,-0.019880,-0.076311
7323,2015-04-29,PFE,Sale (Partial),"$15,000",David Perdue,0.021800,-0.002605,0.039443,0.012301
7325,2015-05-06,PFE,Sale (Partial),"$15,000",David Perdue,0.018828,-0.002690,0.079199,0.056007
7327,2015-05-07,PFE,Sale (Partial),"$15,000",David Perdue,0.015188,-0.009827,0.075042,0.049532


In [22]:
display(ready_csv_data.head(50))

,transaction_date,ticker,type,amount,senator,30_days_return,60_days_return,90_days_return,180_days_return
50,2023-04-03,MSFT,Sale (Partial),"$50,000",Tommy Tuberville,0.059778,0.170258,0.188192,0.139763
293,2022-11-22,MSFT,Purchase,"$15,000",Dan Sullivan,-0.027915,-0.019630,0.055815,0.305289
318,2022-11-30,MSFT,Purchase,"$50,000",Dan Sullivan,-0.060045,-0.027358,-0.019971,0.310862
323,2022-11-23,MSFT,Sale (Full),"$50,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
324,2022-11-23,MSFT,Purchase,"$15,000",Tommy Tuberville,-0.035746,-0.029728,0.023115,0.303370
326,2022-11-22,MSFT,Sale (Partial),"$250,000",Tommy Tuberville,-0.027915,-0.019630,0.055815,0.305289
359,2022-10-28,MSFT,Sale (Full),"$15,000",Tommy Tuberville,0.052222,0.007452,0.054390,0.258932
363,2022-10-24,MSFT,Purchase,"$50,000",Tommy Tuberville,0.004157,-0.031738,-0.025695,0.161913
365,2022-10-13,MSFT,Purchase,"$500,000",Tommy Tuberville,0.054944,0.081035,0.009368,0.213872
402,2022-10-05,MSFT,Sale (Partial),"$250,000",Tommy Tuberville,-0.111597,0.026239,-0.035894,0.158752


In [24]:
ready_csv_data.hvplot.bar(x='transaction_date', y='30_days_return')

:Bars   [transaction_date]   (30_days_return)

In [28]:
ready_ticker_data.to_csv('/Users/matt/Desktop/senator_trading_performance/ready_ticker_data.csv', index=False)